In [ ]:
from google.colab import files
uploaded = files.upload()

# Load Packages

In [ ]:
!pip install --upgrade pingouin

In [ ]:
import pingouin as pg
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import statsmodels.api as sm

from itertools import cycle
from sklearn.metrics import (confusion_matrix, accuracy_score, recall_score, precision_score, f1_score, classification_report)
from sklearn.metrics import roc_curve, auc
from sklearn.model_selection import cross_val_score, train_test_split
from scipy import interp
from sklearn.metrics import roc_auc_score
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler

from sklearn.tree import DecisionTreeClassifier
from sklearn import metrics
from imblearn.over_sampling import SMOTE

# Load the data

In [ ]:
df = pd.read_excel('###.xlsx')

In [ ]:
da = df.drop(columns=['Unnamed: 0'])

In [ ]:
len(da.columns)

# Kolmogorov-Smirnov test for normality

In [ ]:
from numpy.random import seed
from numpy.random import randn
from scipy.stats import Kstest

data = da  #da['EDLengthofStayInHrs']

# normality test
stat, p = Kstest(data)
print('Statistics=%.3f, p=%.3f' % (stat, p))

# interpret
alpha = 0.05
if p > alpha:
	print('Sample looks Gaussian (fail to reject H0)')
else:
	print('Sample does not look Gaussian (reject H0)')

# Defining the dependent and independent variables - based off Spearman Correlation Analysis

In [ ]:
X = da[['ESI_x',	'Physician Available',	'Nurse Available',	'Bed Available',	'Temp',	'RR',	'SpO2', 'HAS_AFIB',	'HAS_CAD', 'HAS_CHRONIC_LUNG_DISEASE',	
        'HAS_CKD', 'HAS_CVD',	'HAS_DIABETES',	'HAS_PERIPH_VASC_DIS', 'HAS_OBESITY', 'PatientRace_Black']]

y = da['EDLOS']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state = 42)

# Applying SMOTE

In [ ]:
print("Before OverSampling, counts of label '1': {}".format(sum(y_train==1)))
print("Before OverSampling, counts of label '0': {} \n".format(sum(y_train==0)))

s = SMOTE(random_state = 42) # SMOTE function for oversampling
X_train_s, y_train_s = s.fit_sample(X_train, y_train.ravel())

print('After OverSampling, the shape of train_X: {}'.format(X_train_s.shape))
print('After OverSampling, the shape of train_y: {} \n'.format(y_train_s.shape))

print("After OverSampling, counts of label '1': {}".format(sum(y_train_s==1)))
print("After OverSampling, counts of label '0': {} \n".format(sum(y_train_s==0)))

# Logistic Regression Model

In [ ]:
from sklearn.linear_model import LogisticRegression
from numpy import mean, std
scikit_default = LogisticRegression(random_state=0).fit(X_train_s, y_train_s)
print(f"intecept: {scikit_default.intercept_} coeficients: {scikit_default.coef_}")
print(f"train accuracy: {scikit_default.score(X_train_s, y_train_s)}")
print(f"test accuracy: {scikit_default.score(X_test, y_test)}")

In [ ]:
# defining the dependent and independent variables
# loading the testing dataset # df = pd.read_csv('logit_test1.csv', index_col = 0)
Xtest = X_test
ytest = y_test

# performing predictions on the test datdaset
yhat = scikit_default.predict(Xtest)
prediction = list(map(round, yhat))
  
# comparing original and predicted values of y
print('Acutal values', list(ytest.values))
print('Predictions :', prediction)

In [ ]:
from sklearn.model_selection import cross_val_score
from sklearn.metrics import classification_report, confusion_matrix
rfc_cv_score = cross_val_score(scikit_default, X_train_s, y_train_s, cv=10, scoring='roc_auc')

# Confusion matrix
print("=== Confusion Matrix ===")
print(confusion_matrix(ytest, prediction))
print('\n')

# Classification report
print("=== Classification Report ===")
print(classification_report(ytest, prediction))
print('\n')

print("=== All AUC Scores ===")

print(rfc_cv_score)
print('\n')

print("=== Mean AUC Score ===")
print("Mean AUC Score - Random Forest: ", rfc_cv_score.mean())
print("Std AUC Score - Random Forest: ", rfc_cv_score.std())

# Gradient Boosting Model

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error

# define dataset
X = da[['ESI_x',	'Physician Available',	'Nurse Available',	'Bed Available',	'Temp',	'RR',	'SpO2', 'HAS_AFIB',	'HAS_CAD', 'HAS_CHRONIC_LUNG_DISEASE',	
        'HAS_CKD', 'HAS_CVD',	'HAS_DIABETES',	'HAS_PERIPH_VASC_DIS', 'HAS_OBESITY', 'PatientRace_Black']]
y = da[['EDLOS']]

# split into train/test sets
Xtrain, Xtest, ytrain, ytest = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
clf = GradientBoostingClassifier(max_depth=2, n_estimators=3, learning_rate=1.0)
clf.fit(X_train_s, y_train_s)

In [ ]:
# optimal number of trees
best_classifier = GradientBoostingClassifier(max_depth=4, n_estimators=2, learning_rate=1.0)
best_classifier.fit(X_train_s, y_train_s)

In [ ]:
# performance evaluation
y_pred = best_classifier.predict(Xtest)

In [ ]:
from sklearn.model_selection import cross_val_score
from sklearn.metrics import classification_report, confusion_matrix
rfc_cv_score = cross_val_score(best_classifier, X_train_s, y_train_s, cv=10, scoring='roc_auc')

# Confusion matrix
print("=== Confusion Matrix ===")
print(confusion_matrix(ytest, y_pred))
print('\n')

# Classification report
print("=== Classification Report ===")
print(classification_report(ytest, y_pred))
print('\n')

print("=== All AUC Scores ===")

print(rfc_cv_score)
print('\n')

print("=== Mean AUC Score ===")
print("Mean AUC Score - Random Forest: ", rfc_cv_score.mean())
print("Std AUC Score - Random Forest: ", rfc_cv_score.std())

# Decision Trees

In [ ]:
feature_cols = ['ESI_x',	'Physician Available',	'Nurse Available',	'Bed Available',	'Temp',	'RR',	'SpO2', 'HAS_AFIB',	'HAS_CAD', 'HAS_OBESITY',
                'HAS_CHRONIC_LUNG_DISEASE',	'HAS_CKD', 'HAS_CVD',	'HAS_DIABETES',	'HAS_PERIPH_VASC_DIS', 'PatientRace_Black']
X = da[['ESI_x',	'Physician Available',	'Nurse Available',	'Bed Available',	'Temp',	'RR',	'SpO2', 'HAS_AFIB',	'HAS_CAD', 'HAS_CHRONIC_LUNG_DISEASE',	
        'HAS_CKD', 'HAS_CVD',	'HAS_DIABETES',	'HAS_PERIPH_VASC_DIS', 'HAS_OBESITY', 'PatientRace_Black']]
y = da[['EDLOS']]

# Split dataset into training and test set
X_train, X_test, y_train, ytest = train_test_split(X, y, test_size=0.2, random_state = 42)

In [ ]:
# Create Decision Tree classifer object
clf = DecisionTreeClassifier()

# Train Decision Tree Classifer
clf = clf.fit(X_train_s, y_train_s)

#Predict the response for test dataset
y_pred = clf.predict(X_test)

In [ ]:
# Model Accuracy
print(classification_report(ytest, y_pred))

In [ ]:
# Visualizing Tree
from sklearn.tree import export_graphviz
from sklearn.externals.six import StringIO  
from IPython.display import Image  
import pydotplus

dot_data = StringIO()
export_graphviz(clf, out_file=dot_data,  
                filled=True, rounded=True,
                special_characters=True,feature_names = feature_cols,class_names=['0','1'])
graph = pydotplus.graph_from_dot_data(dot_data.getvalue())  
graph.write_png('EDLOS.png')
Image(graph.create_png())

In [ ]:
# Optimizing Decision Tree
# Create Decision Tree classifer object
clf = DecisionTreeClassifier(criterion="entropy", max_depth=5) 

# Train Decision Tree Classifer
clv = clf.fit(X_train_s, y_train_s)

#Predict the response for test dataset
y_pred = clv.predict(X_test)

# Model Accuracy
print(classification_report(ytest, y_pred))

In [ ]:
# Visualizing optimal tree
from sklearn.externals.six import StringIO  
from IPython.display import Image  
from sklearn.tree import export_graphviz
import pydotplus
dot_data = StringIO()
export_graphviz(clf, out_file=dot_data,  
                filled=True, rounded=True,
                special_characters=True, feature_names = feature_cols,class_names=['0','1'])
graph = pydotplus.graph_from_dot_data(dot_data.getvalue())  
graph.write_png('EDLOS.png')
Image(graph.create_png())

In [ ]:
from sklearn.model_selection import cross_val_score
from sklearn.metrics import classification_report, confusion_matrix
rfc_cv_score = cross_val_score(clv, X_train_s, y_train_s, cv=10, scoring='roc_auc')

print("=== Confusion Matrix ===")
print(confusion_matrix(y_test, y_pred))
print('\n')

print("=== Classification Report ===")
print(classification_report(y_test, y_pred))
print('\n')

print("=== All AUC Scores ===")

print(rfc_cv_score)
print('\n')

print("=== Mean AUC Score ===")
print("Mean AUC Score - Random Forest: ", rfc_cv_score.mean())
print("Std AUC Score - Random Forest: ", rfc_cv_score.std())

# Random Forest

In [ ]:
# Import train_test_split function
X = da[['ESI_x',	'Physician Available',	'Nurse Available',	'Bed Available',	'Temp',	'RR',	'SpO2', 'HAS_AFIB',	'HAS_CAD', 'HAS_CHRONIC_LUNG_DISEASE',	
        'HAS_CKD', 'HAS_CVD',	'HAS_DIABETES',	'HAS_PERIPH_VASC_DIS', 'HAS_OBESITY', 'PatientRace_Black']]
y = da[['EDLOS']]

# Split dataset into training set and test set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state = 42)

In [ ]:
from sklearn.ensemble import RandomForestClassifier
#Create a Gaussian Classifier
clf = RandomForestClassifier(n_estimators=100, criterion='entropy')

#Train the model using the training sets y_pred=clf.predict(X_test)
clf.fit(X_train_s, y_train_s)

#Predict
ypredt=clf.predict(X_test)

In [ ]:
from sklearn.model_selection import cross_val_score
from sklearn.metrics import classification_report, confusion_matrix
rfc_cv_score = cross_val_score(clf, X_train_s, y_train_s, cv=10, scoring='roc_auc')

# Confusion matrix
print("=== Confusion Matrix ===")
print(confusion_matrix(y_test, ypredt))
print('\n')

# Classification report
print("=== Classification Report ===")
print(classification_report(y_test, ypredt))
print('\n')

print("=== All AUC Scores ===")

print(rfc_cv_score)
print('\n')

print("=== Mean AUC Score ===")
print("Mean AUC Score - Random Forest: ", rfc_cv_score.mean())
print("Std AUC Score - Random Forest: ", rfc_cv_score.std())

# Training Multiple Classifiers and Recording the Results

In [ ]:
# Import the classifiers
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

from sklearn.metrics import roc_curve, roc_auc_score

In [ ]:
# Instantiate the classfiers and make a list
classifiers = [LogisticRegression(random_state = 42), 
               GradientBoostingClassifier(max_depth=4, n_estimators=2, learning_rate=1.0),
               DecisionTreeClassifier(random_state = 42, criterion="entropy", max_depth=5),
               RandomForestClassifier(random_state = 42, n_estimators=100, criterion='entropy')]

# Define a result table as a DataFrame
result_table = pd.DataFrame(columns=['classifiers', 'fpr','tpr','auc'])

In [ ]:
# Train the models and record the results
for cls in classifiers:
    model = cls.fit(X_train_s, y_train_s)
    yproba = model.predict_proba(X_test)[::,1]
    
    fpr, tpr, _ = roc_curve(y_test,  yproba)
    auc = roc_auc_score(y_test, yproba)
    
    result_table = result_table.append({'classifiers':cls.__class__.__name__,
                                        'fpr':fpr, 
                                        'tpr':tpr, 
                                        'auc':auc,}, ignore_index=True)

In [ ]:
# Set name of the classifiers as index labels
result_table.set_index('classifiers', inplace=True)

In [ ]:
fig = plt.figure(figsize=(8,6))

for i in result_table.index:
    plt.plot(result_table.loc[i]['fpr'], 
             result_table.loc[i]['tpr'], 
             label="{}, AUC={:.3f}".format(i, result_table.loc[i]['auc']))
    
plt.plot([0,1], [0,1], color='orange', linestyle='--')

plt.xticks(np.arange(0.0, 1.1, step=0.1))
plt.xlabel("False Positive Rate", fontsize=15)

plt.yticks(np.arange(0.0, 1.1, step=0.1))
plt.ylabel("True Positive Rate", fontsize=15)

plt.title('ROC Curve Analysis for Different Models - SMOTE applied data', fontweight='bold', fontsize=15)
plt.legend(prop={'size':13}, loc='lower right')

plt.show()

In [ ]:
# Operational Point for AUC
from numpy import argmax
from sklearn.metrics import precision_recall_curve
# Calculate roc curves
precision, recall, thresholds = precision_recall_curve(y_test, yproba)
# Convert to f score
fscore = (2 * precision * recall)/(precision + recall)
# Locate the index of the largeest f score
ix = argmax(fscore)
print('Best Threshold=%f, F-score=%.3f' % (thresholds[ix], fscore[ix]))